# Typology for doubles

1. Identify 2-artifact clusters (pairs of contiguous artifacts); make sure that the union has no interior
2. For each cluster, classify the inside edge as C or nonC (`drop_interline`: bool)
3. Visualize and test if it works

- number of nodes
- number of continuity groups
- filter non-planarity artifacts
- CES counts
- prime detection
- crosses detection
- touches detecion

In [2]:
import geopandas as gpd
import matplotlib.pyplot as plt
import momepy
import numpy as np
import pandas as pd
import shapely
from libpysal import graph
from scipy import sparse
import folium
import folium.plugins as plugins
import shapely

from core import algorithms, utils
from core.geometry import voronoi_skeleton

Specify case metadata

In [3]:
case = "Liège"

Read road data

In [4]:
roads = utils.read_original(case)

Remove duplicated roads

In [5]:
roads = momepy.remove_false_nodes(roads)
roads = roads[~roads.geometry.duplicated()].reset_index()
roads = momepy.remove_false_nodes(roads)

Identify artifacts

In [7]:
artifacts = momepy.FaceArtifacts(roads).face_artifacts.set_crs(roads.crs)
artifacts["id"] = artifacts.index

Remove edges fully within the artifact (dangles).

In [8]:
a_idx, _ = roads.sindex.query(artifacts.geometry, predicate="contains")
artifacts = artifacts.drop(artifacts.index[a_idx])

Filter doubles

In [13]:
rook = graph.Graph.build_contiguity(artifacts, rook=True)
artifacts['comp'] = rook.component_labels
counts = artifacts['comp'].value_counts()
artifacts = artifacts.loc[artifacts['comp'].isin(counts[counts==2].index)]

In [40]:
artifacts

,geometry,face_artifact_index,id,node_count,comp,stroke_count,C,E,S,non_planar,non_planar_cluster,interstitial_nodes,ces_type,inter_road,inter_coins_end,drop_interline
10,"POLYGON ((685360.558 5609766.62, 685359.515 56...",7.000197,10,5,3,5,1,3,1,False,0,0,5CEEES,24105,True,True
11,"POLYGON ((682082.999 5609910.253, 682075.805 5...",5.472530,11,4,4,4,3,1,0,False,0,0,4CCCE,7362,False,False
19,"POLYGON ((679864.921 5612652.577, 679838.132 5...",5.477689,19,3,6,3,1,1,1,False,0,0,3CES,15969,True,True
123,"POLYGON ((665538.217 5608558.621, 665611.953 5...",5.875323,123,4,22,3,1,1,1,False,0,1,4CES,116,True,True
124,"POLYGON ((665611.953 5608515.688, 665538.217 5...",5.419501,124,3,22,3,1,1,1,False,0,0,3CES,116,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6689,"POLYGON ((688772.028 5621515.847, 688780.77 56...",3.963123,6689,3,1139,3,1,1,1,False,0,0,3CES,27101,False,False
6750,"POLYGON ((690331.109 5616336.72, 690327.177 56...",5.915152,6750,5,1152,1,0,0,1,False,0,4,5S,27932,False,True
6752,"POLYGON ((690326.665 5616357.732, 690330.555 5...",3.506460,6752,3,1152,3,1,1,1,False,0,0,3CES,27932,False,True
6759,"POLYGON ((690618.224 5615362.332, 690613.569 5...",5.602549,6759,3,584,3,1,1,1,False,0,0,3CES,28007,True,True


Optionally define a subset of data.

In [41]:
data_sample = artifacts.sample(50)

In [42]:
new_roads = simplify_pairs(data_sample, roads)

AssertionError: 

**Classify interlines**

In [39]:
def _classify_interline(mycluster, artifacts, roads):

    # get the cluster geometry
    cluster_geom = artifacts[artifacts.comp == mycluster].union_all()

    # find the road segment that is contained within the cluster geometry
    road_contained = roads.sindex.query(cluster_geom, predicate="contains")

    # make sure we have uniquely identified the road segment
    assert len(road_contained)==1

    # return the ID of the road segment (to potentially drop later) and coins_end True/False
    return road_contained[0], roads.loc[road_contained]["coins_end"].values[0]


def simplify_pairs(artifacts, roads, distance=2):
    # LOOP 0 - drop and treat as a whole
    # TODO: include half on non-dropping

    # Filer artifacts caused by non-planar intersections.
    artifacts["non_planar"] = artifacts["stroke_count"] > artifacts["node_count"]
    a_idx, r_idx = roads.sindex.query(artifacts.geometry.boundary, predicate="overlaps")
    artifacts.iloc[np.unique(a_idx), -1] = True

    # Remove (for now) the clusters that contain at least one non-planar component
    # (TODO: we will deal with them later, ...?)
    # non_planar_cluster: number of non-planar artifacts per cluster
    artifacts["non_planar_cluster"] = artifacts.apply(lambda x: sum(artifacts.loc[artifacts["comp"]==x.comp]["non_planar"]), axis = 1)
    # dealing with the non-planar later!
    artifacts = artifacts[artifacts.non_planar_cluster == 0]

    # Compute number of stroke groups per artifact
    roads, _ = algorithms.common.continuity(roads)
    strokes, c_, e_, s_ = algorithms.common.get_stroke_info(artifacts, roads)

    artifacts["stroke_count"] = strokes
    artifacts["C"] = c_
    artifacts["E"] = e_
    artifacts["S"] = s_

    interlines = artifacts.comp.apply(lambda x: _classify_interline(x, artifacts, roads))
    artifacts["inter_road"] = [v[0] for v in interlines]
    artifacts["inter_coins_end"] = [v[1] for v in interlines]
    artifacts["drop_interline"] = artifacts["inter_coins_end"]

    # if one of the 2 artifacts is of type 0C0EnS, then no matter whether inter_coins_end is True or False! it will always be S
    comps_with_s = list(artifacts[(artifacts.C == 0) & (artifacts.E == 0)].comp)
    artifacts.loc[artifacts["comp"].isin(comps_with_s), "drop_interline"] = True

    to_drop = artifacts.drop_duplicates("comp").query("drop_interline == True").inter_road
    merged_pairs = artifacts.query("drop_interline == True").dissolve("comp")

    roads_cleaned = roads.drop(to_drop.values)

    loop_1 = algorithms.simplify.simplify_singletons(merged_pairs, roads_cleaned, distance=distance)

    # LOOP_2
    # TODO: second half
    return loop_1